In [1]:
#Цикл всегда 80 с переопределением расхода
import prop
import pandas as pd

#Таблица с потоками и блоками
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды
gas     = "REFPROP::WATER"              #Состав газа
Tin     = 300                           #Температура газов на входе
Toutmin = 80                            #Температура газов на выходе
Ggas    = 100                           #Расход газов
Pgas    = 10                            #Давление газов
Hgasin  = prop.t_p(Tin,Pgas,gas)["H"]   #Энтальпия газов на входе

#Параметры ОЦР
fluid = "REFPROP::R236ea"
Ppump = 5                 #Давление на выходе из насоса
Pcond = 0.2               #Давление в конденсаторе
KPDpump = 0.8             #КПД насоса
KPDturb = 0.9             #КПД турбины
G = 100                   #Расход среды (на 1 шаге)
dTmin = 15                #Минимальный напор в регенераторе
dTmin_heater = 20         #Напор в нагревателе

#Внос данных в таблицу
streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,Hgasin,prop.h_p(Hgasin,Pgas,gas)["S"],Ggas,1]

#Начало расчёта - поток перед насосом
streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

#Цикл для сведения с регенератором
for i in range(5):
    #Насос
    from modules import pump
    pump.calc("COND-PUMP","PUMP-REG",Ppump,KPDpump,fluid,streams,blocks)

    #Регенератор
    from modules import regen
    regen.calc("TURB-REG","REG-COND","PUMP-REG","REG-HEAT",dTmin,fluid,streams,blocks)

    #Нагреватель с расчётом расхода
    from modules import heater80
    heater80.calc("IN-HEAT","HEAT-OUT","REG-HEAT","HEAT-TURB",dTmin_heater,Toutmin,gas,fluid,streams,blocks)

    #Турбина
    from modules import turbine
    turbine.calc("HEAT-TURB","TURB-REG",Pcond,KPDturb,fluid,streams,blocks)

    #Регенератор
    from modules import regen
    regen.calc("TURB-REG","REG-COND","PUMP-REG","REG-HEAT",dTmin,fluid,streams,blocks)

    #Конденсатор
    from modules import condenser
    condenser.calc("REG-COND","COND-PUMP",fluid,streams,blocks)

streams

,T,P,H,S,G,Q
IN-HEAT,300.000000,10.0,1343.333966,3.248794,100.00000,1.0
HEAT-OUT,80.000000,10.0,342.935929,1.069080,100.00000,0.0
COND-PUMP,24.163111,0.2,230.020348,1.105003,412.91432,0.0
PUMP-REG,26.554694,5.0,234.199578,1.107795,412.91432,0.0
REG-HEAT,135.626248,5.0,386.029888,1.536542,412.91432,0.0
HEAT-TURB,280.000000,5.0,628.307278,2.054369,412.91432,1.0
TURB-REG,194.033019,0.2,553.599110,2.072285,412.91432,1.0
REG-COND,41.554694,0.2,401.768801,1.681187,412.91432,1.0


In [2]:
blocks

,N,Q
PUMP,1725.663805,NaN
TURBINE,30848.072050,NaN
REGENERATOR,NaN,62692.909060
CONDENSER,NaN,70917.395476
HEATER,NaN,100039.803721


In [3]:
#Баланс
print(blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N'])

#КПД
KPD =(blocks.loc['TURBINE','N']-blocks.loc['PUMP','N'])/ blocks.loc['HEATER','Q']
print(KPD)

-7.275957614183426e-12
0.2911082105526925
